In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [2]:
## Load the dataset
data = pd.read_csv("Churn_Modelling.csv")
print(data.shape)
data.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
## Drop irrelevant features

data=data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)


In [4]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Encode categorical columns
gender_encoder = LabelEncoder()
data['Gender']=gender_encoder.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
import sklearn
print(sklearn.__version__)

In [6]:
## One hot encode - Geography column (To avoid wrong comparisons)

from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [7]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [8]:

onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
## converting the encoded columns as a dataframe

geo_encoded_df=pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
## Replace Geography col with encoded cols
data = pd.concat([data.drop('Geography', axis=1) , geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
## Let's save the Encoders and Scaler as pickle for future use

with open('gender_encoder.pkl', 'wb') as file:
    pickle.dump(gender_encoder, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [12]:
## Divide the dataset into independent and dependent features
X=data.drop('Exited', axis=1)
Y=data['Exited']
X.head()
Y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [13]:
## Split the data into training and testing dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [14]:
## Scale the features using StandardScaler
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)



In [15]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [16]:
###NEXT TRAIN THE ANN ON THE DATASET

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
# import datetime

In [ ]:
# import tensorflow as tf
print("hello world")

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [21]:
X_train.shape[1]

12

In [22]:
## Building ANN model

model=Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 connected to inputs
        Dense(32, activation='relu'), ## HL2 
        Dense(1, activation='sigmoid') ## Output Layer
        
    ]
)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
import tensorflow
opt=tensorflow.keras.optimizers.legacy.Adam(learning_rate=0.005)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [36]:
## Inorder to do Forward/Backward propagation 
## We need to compile the model by providing functions like optimizer/loss/metrics

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [46]:
## Setting up Tensorboard

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir="logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [38]:
## Set up Early Stopping to stop once the loss change is negligible

early_stopping_callback=EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [39]:
## Train the model

history=model.fit(
    X_train,Y_train,validation_data=(X_test,Y_test), epochs=150,
    callbacks=[early_stopping_callback,tensorflow_callback]
)

Epoch 1/150
250/250 [==============================] - 0s 905us/step - loss: 0.3197 - accuracy: 0.8674 - val_loss: 0.3351 - val_accuracy: 0.8635
Epoch 2/150
250/250 [==============================] - 0s 745us/step - loss: 0.3162 - accuracy: 0.8695 - val_loss: 0.3386 - val_accuracy: 0.8615
Epoch 3/150
250/250 [==============================] - 0s 788us/step - loss: 0.3116 - accuracy: 0.8731 - val_loss: 0.3528 - val_accuracy: 0.8590
Epoch 4/150
250/250 [==============================] - 0s 772us/step - loss: 0.3102 - accuracy: 0.8734 - val_loss: 0.3399 - val_accuracy: 0.8615
Epoch 5/150
250/250 [==============================] - 0s 744us/step - loss: 0.3061 - accuracy: 0.8751 - val_loss: 0.3443 - val_accuracy: 0.8615
Epoch 6/150
250/250 [==============================] - 0s 719us/step - loss: 0.3046 - accuracy: 0.8739 - val_loss: 0.3422 - val_accuracy: 0.8610
Epoch 7/150
250/250 [==============================] - 0s 712us/step - loss: 0.3015 - accuracy: 0.8767 - val_loss: 0.3499 - val_ac

In [40]:
model.save('model.h5')

/Users/e151858/Desktop/Projects/Personal/ann_classification/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [42]:
## Load TensorBoard extension

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [45]:
%tensorboard --logdir logs/fit20250822

Reusing TensorBoard on port 6007 (pid 34371), started 0:01:21 ago. (Use '!kill 34371' to kill it.)

In [ ]:
## Load the pickle files

